In [1]:
whos

Interactive namespace is empty.


In [2]:
pwd

'/mmfs1/data/aglinska/BC-ABCD-ADHD'

In [3]:
import ants
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt

/data/aglinska/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/data/aglinska/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/data/aglinska/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [4]:
def get_triu(inMat):
    #inMat = rdm_bg

    assert np.ndim(inMat)==2, 'not 2 dim, wtf'
    assert inMat.shape[0]==inMat.shape[1], 'not a square'

    n = inMat.shape[0]
    triu_vec = inMat[np.triu_indices(n=n,k=1)]

    #assert (squareform(triu_vec)==inMat).sum()/(n**2)>.9, 'unfaithful triu'
    return triu_vec

In [5]:
indir = './Data/ABCD-data-use-T0'
folders = [folder for folder in os.listdir(indir) if folder.startswith('sub-')]
print(len(folders))
folders[0:5]

4836


['sub-NDARINVLBWER7ME',
 'sub-NDARINVTT9JRJDR',
 'sub-NDARINVZ8U8TUHJ',
 'sub-NDARINV9AH5WYA7',
 'sub-NDARINV42LK7GJJ']

In [6]:
fn_temp = './Data/ABCD-data-use-T0/{sub}/ses-baselineYear1Arm1/anat/{sub}_ses-baselineYear1Arm1_run-01_T1w.nii'

In [7]:
#im_temp = ants.image_read(fn_temp.format(sub=folders[0]))

In [8]:
im_temp = ants.resample_image(ants.image_read(fn_temp.format(sub=folders[0])),resample_params=(64,64,64),use_voxels=True)

In [9]:
# 256,256,256 memory hog
#anat_arr = [ants.image_read(fn_temp.format(sub=folders[i])).numpy().astype(np.float16) for i in tqdm(range(100))]

In [10]:
nsubs = len(folders)
print(nsubs)

4836


In [11]:
# 64,64,64 and float16 resample
anat_arr = [ants.resample_image(ants.image_read(fn_temp.format(sub=folders[i])),resample_params=(64,64,64),use_voxels=True).numpy().astype(np.float16) for i in tqdm(range(nsubs))]

  1%|▌                                      | 65/4836 [02:15<2:46:20,  2.09s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
%%time
anat_arr = np.array(anat_arr)

In [ ]:
anat_arr.shape

In [ ]:
anat_arr.dtype

In [ ]:
manat_arr = anat_arr.mean(axis=0)

In [ ]:
im_temp.new_image_like(manat_arr.astype(np.float32)).plot_ortho(flat=True,xyz_lines=False,orient_labels=True,figsize=1.0)

In [ ]:
flat_arr = anat_arr.reshape(nsubs,-1)
print(flat_arr.shape)
flat_arr = flat_arr[:,flat_arr.sum(axis=0)!=0]
print(flat_arr.shape)
#plt.imshow(anat_arr.reshape(100,-1))

In [ ]:
cmat_arr = np.corrcoef(flat_arr)
cmat_arr.shape

In [ ]:
cbar.set_label?

In [ ]:
plt.imshow(cmat_arr,vmin=get_triu(cmat_arr).min(),vmax=get_triu(cmat_arr).max())
cbar = plt.colorbar()
plt.title('Similarity of subject anatomy')
plt.xlabel('Subjects')
plt.ylabel('Subjects')

In [ ]:
mflat_arr = flat_arr.mean(axis=0)

In [ ]:
corr2mean = [np.corrcoef(flat_arr[i,:],mflat_arr)[0,1] for i in tqdm(range(flat_arr.shape[0]))]

In [ ]:
plt.hist(corr2mean)
plt.title('Subject similarity to mean image')
plt.xlabel('correlation with mean image')

In [ ]:
argsort = np.argsort(corr2mean)

In [ ]:
i = len(argsort)-1
ants.image_read(fn_temp.format(sub=folders[argsort[i]])).plot_ortho(flat=True,xyz_lines=False,orient_labels=False,title=f'ID:{argsort[i]} | {folders[argsort[i]]} | corr2mean:{corr2mean[argsort[i]]:.2f}',figsize=1.2,titlefontsize=20)

In [ ]:
for i in range(5):
    ants.image_read(fn_temp.format(sub=folders[argsort[i]])).plot_ortho(flat=True,xyz_lines=False,orient_labels=False,title=f'ID:{argsort[i]} | {folders[argsort[i]]} | corr2mean:{corr2mean[argsort[i]]:.2f}',figsize=1.2,titlefontsize=20)